In [2]:
%load_ext blackcellmagic

import tqdm
import numpy as np
import pandas as pd

import bokeh.io
import bokeh.plotting

import bokeh_catplot

bokeh.io.output_notebook()

Loading BokehJS ...

# Exercise 8.1: Writing functions for bootstrap replicates

a) In the lessons, we wrote a function, draw_bs_rep() to draw a single bootstrap replicate out of a single set of repeated measurements. Update this function to have a size keyword argument so that you can draw many bootstrap replicates and return a Numpy array of the replicates. Here are step-by-step instructions.

1. Define a function with call signature `draw_bs_reps(data, func, rg, size=1, args=())`, where `func` is a function that takes in an array and returns a statistic; it has call signature `func(data, *args)`. Examples that could be passed in as `func` are `np.mean`, `np.std`, `np.median`, or a user-defined function. `rg` is an instance of a Numpy random number generator. `size` is the number of replicates to generate.
1. Write a good doc string.
1. Define n to be the length of the input data array.
1. Use a list comprehension to compute a list of bootstrap replicates.
1. Return the replicates as a Numpy array.


In [3]:
def draw_bs_reps(data, func, rg=np.random.default_rng(), size=1, args=()):
    """
    Draws bootstrap replicates out of a single set of repeated measurements.
    Function inputs:
    `data`: a NumPy array of data
    `func`: a function to apply to the data array that returns a statistic
    `rg`: an instance of a NumPy random number generator. Defaults to the NumPy default RNG.
    `size`: the number of replicates to generate. Defaults to 1.
    `args`: arguments to pass to `func`.
    
    Returns a NumPy array of bootstrap replicates.
    """
    
    n = len(data)
    
    return np.array([func(rg.choice(data, size=n, replace=True), *args) for _ in range(size)])
    

In [4]:
# Try the function out

example = np.array([1, 2, 3, 4, 5])

draw_bs_reps(example, np.mean, size=10)

array([3.6, 4. , 3.4, 2.6, 2.6, 1.6, 2.6, 2.4, 3.2, 3. ])

b) Write a function analogous to the one in part (a) except for pairs bootstrap. The call signature should be `draw_bs_pairs(data1, data2, func, rg, size=1, args=())`, where `func` has call signature `func(data1, data2, *args)`.

In [5]:
def draw_bs_pairs(data1, data2, func, rg=np.random.default_rng(), size=1, args=()):
    """
    Draws pairs of data points out of `data1` and `data2` and uses them to
    compute bootstrap replicates. Assumes `data1` and `data2` are the same length.
    Function inputs:
    `data`: a NumPy array of data
    `func`: a function to apply to the data array that returns a statistic
    `rg`: an instance of a NumPy random number generator. Defaults to the NumPy default RNG.
    `size`: the number of replicates to generate. Defaults to 1.
    `args`: arguments to pass to `func`.
    
    Returns a NumPy array of bootstrap pairs.
    """
    
    n = len(data1)
    
    bs_reps = np.empty(size)
    
    for i in range(size):
        ind = rg.choice(np.arange(n), size=1)
        a1, a2 = data1[ind], data2[ind]
        bs_reps[i] = func(a1, a2, *args)
        
    return bs_reps

In [6]:
# Try the function out

data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([2, 6, 6, 12, 10])

def ratio(x, y):
    """Computes the ratio of `x` and `y`."""
    return x / y

draw_bs_pairs(data1, data2, ratio, size=5)

array([0.33333333, 0.33333333, 0.5       , 0.5       , 0.33333333])

I wrote a module bootstrap.py to include these two functions.

In [7]:
# Try importing bootstrap.py and using the functions.

import bootstrap

data1 = np.array([1, 2, 3, 4, 5])
data2 = np.array([2, 6, 6, 12, 10])

def ratio(x, y):
    """Computes the ratio of `x` and `y`."""
    return x / y

bootstrap.draw_bs_pairs(data1, data2, ratio, size=5)

array([0.5       , 0.33333333, 0.33333333, 0.5       , 0.33333333])

In [8]:
example = np.array([1, 2, 3, 4, 5])

bootstrap.draw_bs_reps(example, np.mean, size=5)

array([1.8, 3.2, 3.4, 2.8, 3. ])

# Exercise 8.2: Hacker stats with bee sperm data

We will look at the weight of drones (male bees) using the data set stored in `~/git/bootcamp/data/bee_weight.csv` and the sperm quality of drone bees using the data set stored in `~/git/bootcamp/data/bee_sperm.csv`.

a) Load the drone weight data in as a Pandas `DataFrame`. Note that the unit of the weight is milligrams (mg).

In [9]:
!head data/bee_weight.csv

# Data from Straub L, Villamar-Bouza L, Bruckner S, Chantawannakul P, Gauthier L, Khongphinitbunjong K, Retschnig G, Troxler A, Vidondo-Curras B, Neumann P, Williams GR (2016) Neonicotinoid insecticides can serve as inadvertent insect contraceptives. Proceedings of the Royal Society B 283(1835): 20160506. http://dx.doi.org/10.1098/rspb.2016.0506
# Data from Dryad repository, Straub L, Villamar-Bouza L, Bruckner S, Chantawannakul P, Gauthier L, Khongphinitbunjong K, Retschnig G, Troxler A, Vidondo-Curras B, Neumann P, Williams GR (2016) Data from: Neonicotinoid insecticides can serve as inadvertent insect contraceptives. Dryad Digital Repository. http://dx.doi.org/10.5061/dryad.bs515
# Taken from the Excel file from the "Weight Drones" sheet.
Specimen,Colony,Cage,Sample-Nr.,Weight,Treatment,TreatmentNCSS
1,3,1,1,292,Control,1
2,3,1,2,296,Control,1
3,3,1,3,298,Control,1
4,3,1,4,290,Control,1
5,3,1,5,304,Control,1
6,3,1,6,293,Control,1


In [10]:
!head data/bee_sperm.csv

# Data from Straub L, Villamar-Bouza L, Bruckner S, Chantawannakul P, Gauthier L, Khongphinitbunjong K, Retschnig G, Troxler A, Vidondo-Curras B, Neumann P, Williams GR (2016) Neonicotinoid insecticides can serve as inadvertent insect contraceptives. Proceedings of the Royal Society B 283(1835): 20160506. http://dx.doi.org/10.1098/rspb.2016.0506
# Data from Dryad repository, Straub L, Villamar-Bouza L, Bruckner S, Chantawannakul P, Gauthier L, Khongphinitbunjong K, Retschnig G, Troxler A, Vidondo-Curras B, Neumann P, Williams GR (2016) Data from: Neonicotinoid insecticides can serve as inadvertent insect contraceptives. Dryad Digital Repository. http://dx.doi.org/10.5061/dryad.bs515
# Taken from the Excel file from the "Drone Sperm Data" sheet.
Specimen,Treatment,Environment,TreatmentNCSS,Sample ID,Colony,Cage,Sample,Sperm Volume per 500 ul,Quantity,ViabilityRaw (%),Quality,Age (d),Infertil,AliveSperm,Quantity Millions,Alive Sperm Millions,Dead Sperm Millions
227,Control,Cage,1,C2-1-1,

In [11]:
df_weight = pd.read_csv('data/bee_weight.csv', comment='#')
df_weight = df_weight.rename(columns={'Weight':'Weight (mg)'})

df_weight.head()

Specimen  Colony  Cage  Sample-Nr.  Weight (mg) Treatment  TreatmentNCSS
0         1       3     1           1        292.0   Control              1
1         2       3     1           2        296.0   Control              1
2         3       3     1           3        298.0   Control              1
3         4       3     1           4        290.0   Control              1
4         5       3     1           5        304.0   Control              1

b) Plot ECDFs of the drone weight for control and also for those exposed to pesticide. Do you think there is a clear difference?

In [12]:
p = bokeh_catplot.ecdf(
    data=df_weight,
    cats=['Treatment'],
    val='Weight (mg)'
)

bokeh.io.show(p)

There doesn't seem to be a clear difference.

c) Compute the mean drone weight for control and those exposed to pesticide. Compute 95% bootstrap confidence intervals on the mean.

In [13]:
# Get the weight values for control and pesticide
weight_ctrl = df_weight.loc[df_weight['Treatment']=='Control', 'Weight (mg)'].values
weight_pesticide = df_weight.loc[df_weight['Treatment']=='Pesticide', 'Weight (mg)'].values

# Get the means

np.mean(weight_ctrl), np.mean(weight_pesticide)

(277.0563, 278.27333333333337)

In [14]:
weight_ctrl_bs_reps = bootstrap.draw_bs_reps(weight_ctrl, np.mean, size=5000)

print("Mean weight of control bees: ", np.mean(weight_ctrl))
print(
    "95% CI for mean weight of control bees: ",
    np.percentile(weight_ctrl_bs_reps, [2.5, 97.5]),
)

weight_pesticide_bs_reps = bootstrap.draw_bs_reps(weight_pesticide, np.mean, size=5000)

print("Mean weight of pesticide-treated bees: ", np.mean(weight_pesticide))
print(
    "95% CI for mean weight of pesticide-treated bees: ",
    np.percentile(weight_pesticide_bs_reps, [2.5, 97.5]),
)

Mean weight of control bees:  277.0563
95% CI for mean weight of control bees:  [274.7880825  279.37207125]
Mean weight of pesticide-treated bees:  278.27333333333337
95% CI for mean weight of pesticide-treated bees:  [275.045625   281.48922917]


d) Repeat parts (a)-(c) for drone sperm. Use the `'Quality'` column as your measure. This is defined as the percent of sperm that are alive in a 500 µL sample.

In [15]:
df_sperm = pd.read_csv('data/bee_sperm.csv', comment='#', na_values='')

df_sperm.head()

Specimen Treatment Environment  TreatmentNCSS Sample ID  Colony  Cage  \
0       227   Control        Cage              1    C2-1-1       2     1   
1       228   Control        Cage              1    C2-1-2       2     1   
2       229   Control        Cage              1    C2-1-3       2     1   
3       230   Control        Cage              1    C2-1-4       2     1   
4       231   Control        Cage              1    C2-1-5       2     1   

   Sample  Sperm Volume per 500 ul  Quantity  ViabilityRaw (%)    Quality  \
0       1                  2150000   2150000  96.7263814616756  96.726381   
1       2                  2287500   2287500  96.3498079760595  96.349808   
2       3                    87500     87500             98.75  98.750000   
3       4                  1875000   1875000  93.2874208336941  93.287421   
4       5                  1587500   1587500  97.7925061050061  97.792506   

   Age (d)  Infertil  AliveSperm  Quantity Millions  Alive Sperm Millions  \
0       14         0     2079617             2.1500              2.079617   
1       14         0     2204001             2.2875              2.204001   
2       14         0       86406             0.0875              0.086406   
3       14         0     1749139             1.8750              1.749139   
4       14         0     1552456             1.5875              1.552456   

   Dead Sperm Millions  
0             0.070383  
1             0.083499  
2             0.001094  
3             0.125861  
4             0.035044

In [16]:
p = bokeh_catplot.ecdf(
    data=df_sperm,
    cats=['Treatment'],
    val='Quality',
)

p.legend.location = 'top_left'

bokeh.io.show(p)

In [17]:
# Get the sperm quality for control and pesticide-treated bees
quality_ctrl = df_sperm.loc[df_sperm['Treatment']=='Control', 'Quality'].values
quality_pesticide = df_sperm.loc[df_sperm['Treatment']=='Pesticide', 'Quality'].values

np.nanmean(quality_ctrl), np.nanmean(quality_pesticide)

(87.05217771780246, 78.22979590221453)

In [18]:
quality_ctrl_bs_reps = bootstrap.draw_bs_reps(quality_ctrl, np.nanmean, size=5000)

print('Mean sperm quality in control bees: ', np.nanmean(quality_ctrl))

print(
    "95% CI for mean sperm quality in control bees: ",
    np.percentile(quality_ctrl_bs_reps, [0.25, 97.5]),
)

quality_pesticide_bs_reps = bootstrap.draw_bs_reps(
    quality_pesticide, np.nanmean, size=5000
)

print('Mean sperm quality in pesticide-treated bees: ', np.nanmean(quality_pesticide))

print(
    "95% CI for mean sperm quality in pesticide-treated bees: ",
    np.percentile(quality_pesticide_bs_reps, [0.25, 97.5]),
)

Mean sperm quality in control bees:  87.05217771780246
95% CI for mean sperm quality in control bees:  [82.77447487 89.51121471]
Mean sperm quality in pesticide-treated bees:  78.22979590221453
95% CI for mean sperm quality in pesticide-treated bees:  [72.15238016 82.05180601]


e) As you have seen in your analysis in part (d), both the control and pesticide treatments have some outliers with very low sperm quality. This can tug heavily on the mean. So, get 95% bootstrap confidence intervals for the *median* sperm quality of the two treatments.

In [19]:
quality_ctrl_bs_reps_med = bootstrap.draw_bs_reps(quality_ctrl, np.nanmedian, size=5000)
quality_pesticide_bs_reps_med = bootstrap.draw_bs_reps(
    quality_pesticide, np.nanmedian, size=5000
)

print("Median sperm quality in control bees: ", np.nanmedian(quality_ctrl))
print(
    "95% CI for median sperm quality in control bees: ",
    np.percentile(quality_ctrl_bs_reps_med, [0.25, 97.5]),
)

print(
    "Median sperm quality in pesticide-treated bees: ", np.nanmedian(quality_pesticide)
)
print(
    "95% CI for median sperm quality in control bees: ",
    np.percentile(quality_pesticide_bs_reps_med, [0.25, 97.5]),
)

Median sperm quality in control bees:  91.9125976104989
95% CI for median sperm quality in control bees:  [88.5826203 94.2353021]
Median sperm quality in pesticide-treated bees:  83.59405060145176
95% CI for median sperm quality in control bees:  [77.67358997 85.86963819]


# Exercise 8.3: Bootstrapping "theory" with hacker stats

Say we have a data set with $n$ unique measurements. It can be shown that on average a fraction of $(1-1/n)^n$ of the measurements do not appear in a bootstrap sample. Note that for large samples, this is approximately $1/e \approx 1/2.7$, since

$$\lim_{n \to \infty}(1-1/n)^n=1/e.$$

Use hacker stats to show that this is, indeed true. Hint: Think about a convenient “data set” to use for drawing samples.

In [20]:
data = np.arange(100)

rg = np.random.default_rng()

# Make a bootstrap sample

bs_sample = rg.choice(data, size=len(data), replace=True)

bs_sample

array([58, 15, 14, 64, 66, 28, 12, 85, 15, 91, 69, 31, 45, 94, 86, 50, 74,
       40, 93, 21, 22, 32,  5, 12, 90, 95, 82, 22,  9, 21, 27, 91, 61, 69,
       78, 48, 48, 38, 66, 41, 64, 12, 50, 37, 77, 85, 37, 32, 28, 58, 14,
       73, 56, 84, 60,  2, 56, 99, 58, 85, 20, 84, 75, 66, 68,  9, 43, 60,
       99, 30, 75, 77, 98, 48, 67, 35, 16, 69, 86,  6, 56, 86, 20, 88, 27,
       52, 56, 78, 70, 92, 89,  2, 52, 39, 31, 18,  2, 40, 77, 29])

We want to see what samples are in data but not in the bootstrap sample.

In [21]:
not_in = [val for val in data if val not in bs_sample]

len(not_in)

42

In [22]:
# Make and count many bootstrap samples

# Number of measurements in dataset
n = len(data)


# Number of bootstrap samples
n_bs = 10000

frac_not_ins = np.zeros(n_bs)

for i in range(n_bs):
    bs_sample = rg.choice(data, size=n, replace=True)
    not_in = [val for val in data if val not in bs_sample]
    frac_not_ins[i] = len(not_in) / n
    

In [23]:
p = bokeh_catplot.ecdf(
    frac_not_ins,
    title=str(n) + ' measurements, ' + str(n_bs) + ' bootstrap samples'
)

bokeh.io.show(p)

In [24]:
print('Average fraction of measurements that do not appear in a bootstrap sample: ', np.mean(frac_not_ins))
print('95% CI: ', np.percentile(frac_not_ins, [2.5, 97.5]))

Average fraction of measurements that do not appear in a bootstrap sample:  0.366674
95% CI:  [0.31 0.43]


In [25]:
(1 - 1 / n)**n, 1 / np.e

(0.3660323412732292, 0.36787944117144233)

How does the average fraction of measurements that do not appear in the bootstrap sample vary with the number of measurements $n$?

In [27]:
rg = np.random.default_rng()

x = [1, 10, 100, 1000, 10000]
y = np.zeros(len(x))

for j, n in tqdm.tqdm(enumerate(x)):
    data = np.arange(n)
    
    # Number of bootstrap samples
    n_bs = 200

    frac_not_ins = np.zeros(n_bs)

    for i in tqdm.tqdm(range(n_bs)):
        bs_sample = rg.choice(data, size=n, replace=True)
        not_in = [val for val in data if val not in bs_sample]
        frac_not_ins[i] = len(not_in) / n
        
    y[j] = np.mean(frac_not_ins)

0it [00:00, ?it/s]
100%|██████████| 200/200 [00:00<00:00, 12918.27it/s]

100%|██████████| 200/200 [00:00<00:00, 12773.69it/s]

100%|██████████| 200/200 [00:00<00:00, 2457.57it/s]
3it [00:00, 22.60it/s]
100%|██████████| 200/200 [00:01<00:00, 189.15it/s]
4it [00:01,  2.86it/s]
100%|██████████| 200/200 [00:18<00:00, 11.09it/s]
5it [00:19,  3.85s/it]


In [28]:
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Number of measurements',
    y_axis_label='Avg. frac. of measurements not included',
    x_axis_type='log',
)

p.circle(
    x,
    y,
)

# Theoretical

x_theor = np.logspace(0, 4, 400)
y_theor = (1 - 1/x_theor)**x_theor

p.line(
    x_theor,
    y_theor,
    color='orange'
)

bokeh.io.show(p)

In [32]:
%load_ext watermark
%watermark -v -p numpy,pandas,bokeh,bokeh_catplot,tqdm,jupyterlab

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
pandas 0.24.2
bokeh 2.0.2
bokeh_catplot 0.1.7
tqdm 4.46.1
jupyterlab 1.2.6
